# Scraping news from the web

In [1]:
from data.articles import load_news_articles

In [2]:
articles = load_news_articles()

Loading articles from https://www.vg.no/: 100%|██████████| 161/161 [00:58<00:00,  2.74it/s]
Loading articles from https://www.aftenposten.no/: 100%|██████████| 76/76 [00:32<00:00,  2.37it/s]
Loading articles from https://www.nettavisen.no/: 100%|██████████| 24/24 [00:17<00:00,  1.40it/s]


In [3]:
articles["clean_text"] = articles["text"].apply(lambda x: x.lower())
articles["clean_text"] = articles["clean_text"].apply(lambda x: x.replace("\n", " "))
articles["clean_text"] = articles["clean_text"].apply(
    lambda x: " ".join([word for word in x.split() if word.isalpha()]))

In [4]:
articles

,source,url,title,text,clean_text
0,nrk,https://www.nrk.no/ytring/apotekene-er-et-lavt...,Apotekene er et lavterskel helsetilbud,Apotekenes hovedformål er å sikre hele befolkn...,apotekenes hovedformål er å sikre hele befolkn...
1,nrk,https://www.nrk.no/nordland/meloy-kommune-star...,Innbyggerne har landets billigste strøm – men ...,Innerst i Holandsfjorden i nordlandskommunen M...,innerst i holandsfjorden i nordlandskommunen m...
2,aftenposten,https://www.aftenposten.no/oslo/i/47w3A6/krise...,Krisemåling for Raymond Johansen. Taper velger...,Krisemåling for Raymond Johansen. Taper velger...,krisemåling for raymond taper velgere i begge ...
3,nrk,https://www.nrk.no/sorlandet/58-pasienter-har-...,58 pasienter har fått erstatning etter å ha bl...,Hele 58 av de 160 pasientene som har krevd ers...,hele av de pasientene som har krevd erstatning...
4,nrk,https://www.nrk.no/urix/zelenskyj_-10-namngitt...,Zelenskyj: 10 namngitte krigsforbrytarar ident...,Ifølgje presidenten tilhøyrer dei 10 mistenkte...,ifølgje presidenten tilhøyrer dei mistenkte de...
...,...,...,...,...,...
200,nrk,https://www.nrk.no/nordland/studiene-ingen-sok...,Dette er det eneste studietilbudet i Norge med...,Vil du bli maskiningeniør kan du velge mellom ...,vil du bli maskiningeniør kan du velge mellom ...
201,aftenposten,https://www.aftenposten.no/meninger/kommentar/...,Folket som sluttet å snakke sammen,Folket som sluttet å snakke sammen\n\n28. apr....,folket som sluttet å snakke sammen sist oppdat...
202,vg,https://www.vg.no/nyheter/innenriks/i/Bj9WM7/m...,Mæland om karantenehotell: − På grensen av hva...,KARANTENEHOTELL: Quality Airport Hotel Garderm...,quality airport hotel mæland om på grensen av ...
203,vg,https://www.vg.no/forbruker/i/ML83q5/18-feil-d...,18 feil du lærer av porno,18 feil du lærer av porno\n\nSett noe i porno ...,feil du lærer av porno sett noe i porno og ten...


### Loading tokenizer

In [5]:
from data.norec.dataloader import load_full_dataset
from data.dataloader import load_data

dataset = load_full_dataset()
data = load_data(dataset)

maxlen = data['maxlen']
tokenizer = data['tokenizer']

Cleaning text...
Fitting tokenizer...
Converting texts to sequences...
Padding sequences...
One-hot encoding labels...
Done!


# Predicting sentiment of news articles

In [6]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model


def get_sentiment(text):
    # Loading trained model
    model = load_model('../training/models/best_model_0.807860255241394.h5')

    # Preprocessing text
    sequenced = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequenced, maxlen=maxlen)

    # Predicting sentiment
    prediction = model.predict(padded)
    sentiments = ['negative', 'neutral', 'positive']
    return sentiments[np.argmax(prediction)]


articles['sentiment'] = articles['clean_text'].apply(get_sentiment)

In [15]:
articles.head()

,source,url,title,text,clean_text,sentiment
0,nrk,https://www.nrk.no/ytring/apotekene-er-et-lavt...,Apotekene er et lavterskel helsetilbud,Apotekenes hovedformål er å sikre hele befolkn...,apotekenes hovedformål er å sikre hele befolkn...,positive
1,nrk,https://www.nrk.no/nordland/meloy-kommune-star...,Innbyggerne har landets billigste strøm – men ...,Innerst i Holandsfjorden i nordlandskommunen M...,innerst i holandsfjorden i nordlandskommunen m...,neutral
2,aftenposten,https://www.aftenposten.no/oslo/i/47w3A6/krise...,Krisemåling for Raymond Johansen. Taper velger...,Krisemåling for Raymond Johansen. Taper velger...,krisemåling for raymond taper velgere i begge ...,positive
3,nrk,https://www.nrk.no/sorlandet/58-pasienter-har-...,58 pasienter har fått erstatning etter å ha bl...,Hele 58 av de 160 pasientene som har krevd ers...,hele av de pasientene som har krevd erstatning...,neutral
4,nrk,https://www.nrk.no/urix/zelenskyj_-10-namngitt...,Zelenskyj: 10 namngitte krigsforbrytarar ident...,Ifølgje presidenten tilhøyrer dei 10 mistenkte...,ifølgje presidenten tilhøyrer dei mistenkte de...,positive


In [16]:
# Plot the number of sentiments
articles.sentiment.value_counts()

positive    138
neutral      66
negative      1
Name: sentiment, dtype: int64

In [23]:
positive_article = articles[articles.sentiment == 'positive'].head(1).index
positive_article_source = articles.loc[positive_article[0], 'source']
positive_article_title = articles.loc[positive_article, 'title'].values[0]
positive_article_url = articles.loc[positive_article, 'url'].values[0]
positive_article_content = articles.loc[positive_article, 'text'].values[0]

print("Source: {}".format(positive_article_source))
print("Title: {}".format(positive_article_title))
print("URL: {}".format(positive_article_url))
print("Article content:", end="\n\n")
print(positive_article_content)

Source: nrk
Title: Apotekene er et lavterskel helsetilbud
URL: https://www.nrk.no/ytring/apotekene-er-et-lavterskel-helsetilbud-1.15939021
Article content:

Apotekenes hovedformål er å sikre hele befolkningen tilgang på trygge, kvalitetssikrede legemidler.

De skal også gi veiledning om bruken av legemidler, og bidra til sikker og riktig legemiddelbruk.

Apotekfarmasøytene gir uvurderlige bidrag til dette hver eneste dag.

Det er det korte svaret på spørsmålet fra Kaveh Rashidi i NRK Ytring 27. april, om hva hovedformålet til apotek er.

Det er bred politisk oppslutning om det apoteksystemet vi har i Norge, og mange undersøkelser viser at befolkningen er veldig fornøyd med apotekene.

Det er legitimt og fornuftig å stille spørsmål ved om alle varer i apotek er like godt begrunnet, og dette er vurderinger som apotekeierne gjør fortløpende.

Noen handelsvarer kommer og noen går, uten at jeg kan gå inn på hver enkelt vurdering som gjøres.

Det er veldig mange leverandører som ønsker å sel

In [24]:
negative_article = articles[articles.sentiment == 'negative'].head(1).index
negative_article_source = articles.loc[negative_article[0], 'source']
negative_article_title = articles.loc[negative_article, 'title'].values[0]
negative_article_url = articles.loc[negative_article, 'url'].values[0]
negative_article_content = articles.loc[negative_article, 'text'].values[0]

print("Source: {}".format(negative_article_source))
print("Title: {}".format(negative_article_title))
print("URL: {}".format(negative_article_url))
print("Article content:", end="\n\n")
print(negative_article_content)

Source: nrk
Title: Hun er en av bare tre i Norge
URL: https://www.nrk.no/innlandet/laila-fra-valdres-er-en-av-tre-kvinnelige-brannsjefer-i-norge-1.15942900
Article content:

– Vi har kommet så langt i likestillingen at det ikke skulle være noe vanskeligere for kvinner å ha denne jobben enn andre jobber, sier hun.

Men det er langt frem.

Den gjennomsnittlige brannkonstabelen er en etnisk norsk mann i 40-årene som ofte er håndverker ved siden av jobben i brannvesenet.

Bare 3,2 prosent er kvinner.

– Vi trenger et mangfold både i etnisitet, kjønn og seksuell legning, sier brannsjefen i Valdres.

Av 100 brannsjefer i landet er det bare i Valdres, Lofoten og Sandefjord som har kvinnelige brannsjefer.

I neste uke skal alle brannsjefene i Norge undertegne en avtale som forplikter dem til å øke mangfoldet i bransjen.

Fra montør til brannsjef

Laila Lien Østgård kom inn i brannvesenet gjennom jobben som telemontør da hun jobba med brannvarslingsanlegg på gårder.

– Det var interessant å job

In [25]:
neutral_article = articles[articles.sentiment == 'neutral'].head(1).index
neutral_article_source = articles.loc[neutral_article[0], 'source']
neutral_article_title = articles.loc[neutral_article, 'title'].values[0]
neutral_article_url = articles.loc[neutral_article, 'url'].values[0]
neutral_article_content = articles.loc[neutral_article, 'text'].values[0]

print("Source: {}".format(neutral_article_source))
print("Title: {}".format(neutral_article_title))
print("URL: {}".format(neutral_article_url))
print("Article content:", end="\n\n")
print(neutral_article_content)

Source: nrk
Title: Innbyggerne har landets billigste strøm – men for kommunen er det krise
URL: https://www.nrk.no/nordland/meloy-kommune-star-i-fare-for-a-havne-pa-robek-lista-pa-grunn-av-billig-kraft-1.15946413
Article content:

Innerst i Holandsfjorden i nordlandskommunen Meløy ligger Svartisen vannkraftverk.

Her produseres årlig 2195,5 GWh, blant annet fra smeltevann fra Norges nest største isbre, som strekker seg gjennom kommunene Rana, Meløy og Rødøy.

Dette enorme kraftverket har de siste årene bidratt til å produsere Europas billigste strøm.

Men mens strømkunder i Nord-Norge jubler over nær «gratis» strøm, er prisene dårlig nytt for dem som skal tjene penger på strømmen.

De lave strømprisene går nemlig hardt utover kraftkommunenes inntekter fra salg av konsesjonskraft.

Kritisk for Meløy

Lave strømpriser førte til at kraftkommunen Meløy i fjor fikk 20 millioner mindre i inntekter.

– Med de strømprisene som har vært i Nord-Norge de siste to siste årene, har det vært utfordr